# Federated Learning with a Trusted Aggregator

What is a Trusted Aggregator?
> A neutral 3rd party who has a machine that we can trust to not look at the gradients when performing the aggregation.

In [1]:
import syft as sy
import torch as th
hook = sy.TorchHook(th)
from torch import nn, optim

W1024 15:29:08.413697 4405509440 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/Users/jee/.pyenv/versions/miniconda3-latest/envs/pysyft/lib/python3.7/site-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W1024 15:29:08.468665 4405509440 deprecation_wrapper.py:119] From /Users/jee/.pyenv/versions/miniconda3-latest/envs/pysyft/lib/python3.7/site-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [36]:
# create a couple workers
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")


In [37]:
# Toy dataset
data = th.tensor([[0,0], [0,1], [1,0], [1,1.]], requires_grad=True)
target = th.tensor([[0],[0], [1], [1.]], requires_grad=True)

In [38]:
# get pointers to training data on each worker by
# sending some training data to bob and alice.
bobs_data = data[:2].send(bob)
bobs_target = target[:2].send(bob)

alices_data = data[2:].send(alice)
alices_target = data[2:].send(alice)

In [39]:
# init the toy model
model = nn.Linear(2,1)

In [43]:
# instead of having one model like before,
# we want to have two different models that will later be averaged.
bobs_model = model.copy().send(bob)
alices_model = model.copy().send(alice)

In [48]:
bob._objects

{41938895735: tensor([[0., 0.],
         [0., 1.]], requires_grad=True), 63263654675: tensor([[0.],
         [0.]], requires_grad=True), 6909102407: Parameter containing:
 tensor([[ 0.6751, -0.0064]], requires_grad=True), 62370230873: Parameter containing:
 tensor([0.0039], requires_grad=True), 18966693568: tensor([[0., 0.],
         [0., 1.]], requires_grad=True), 51101336522: tensor([[0.],
         [0.]], requires_grad=True), 54865514165: Parameter containing:
 tensor([[0.6258, 0.5383]], requires_grad=True), 53332109346: Parameter containing:
 tensor([-0.1171], requires_grad=True), 64061987705: tensor([[0.0318],
         [0.7127]], grad_fn=<AddmmBackward>), 29840311308: Parameter containing:
 tensor([[0.6258, 0.6808]], requires_grad=True), 70799021953: Parameter containing:
 tensor([0.0318], requires_grad=True)}

In [49]:
bobs_optimizer = optim.SGD(params=bobs_model.parameters(), lr=0.1)
alices_optimizer = optim.SGD(params=alices_model.parameters(), lr=0.1)

In [50]:

for i in range(10):
    # bob
    bobs_opt.zero_grad()
    bobs_pred = bobs_model(bobs_data)
    bobs_loss = ((bobs_pred - bobs_target) ** 2).sum()
    bobs_loss.backward()

    bobs_optimizer.step()
    bobs_loss = bobs_loss.get().data
    
    # alice
    alices_optimizer.zero_grad()
    alices_pred = alices_model(alices_data)
    alices_loss = ((alices_pred - alices_target) ** 2).sum()
    alices_loss.backward()
    
    alices_opt.step()
    alices_loss = alices_loss.get().data

print(bobs_loss)
print(bobs_loss)

KeyError: 'Object "86090934543" not found on worker!!!You just tried to interact with an object ID:86090934543 on <VirtualWorker id:secure_worker #objects:2> which does not exist!!! Use .send() and .get() on all your tensors to make sure they\'reon the same machines. If you think this tensor does exist, check the ._objects dictionaryon the worker and see for yourself!!! The most common reason this error happens is because someone calls.get() on the object\'s pointer without realizing it (which deletes the remote object and sends it to the pointer). Check your code to make sure you haven\'t already called .get() on this pointer!!!'

In [51]:
alices_model.move(secure_worker)
bobs_model.move(secure_worker)

In [52]:
secure_worker._objects

{71428333741: Parameter containing:
 tensor([0.2286], requires_grad=True), 47892080389: tensor([[0.2709],
         [0.4990]], grad_fn=<TBackward>), 76708092899: Parameter containing:
 tensor([[0.6258, 0.6808]], requires_grad=True), 14952959495: Parameter containing:
 tensor([0.0318], requires_grad=True), 60413178618: Parameter containing:
 tensor([[0.6258, 0.5383]], requires_grad=True), 3851722808: Parameter containing:
 tensor([-0.1171], requires_grad=True)}

In [57]:
model.weight.data.set_(((alices_model.weight.data + bobs_model.weight.data) /2).get())
model.bias.data.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())

RuntimeError: set_storage is not allowed on Tensor created from .data or .detach()